<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Data_Augmentation_and_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
!7z x /content/Parasite.7z

--2021-04-17 07:28:41--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Parasite.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20003284 (19M) [application/octet-stream]
Saving to: ‘Parasite.7z’

Parasite.7z         100%[===================>]  19.08M  33.2MB/s    in 0.6s    

2021-04-17 07:28:42 (33.2 MB/s) - ‘Parasite.7z’ saved [20003284/20003284]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 20003284 bytes (20 MiB)

Extracting archive: /content/Parasite.7z
--
Path = /content

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

import torchvision
from PIL import Image

In [3]:
data = pd.read_csv("/content/Dataset/all_meta_data_updated.csv")

In [ ]:
data.species.value_counts()

Plasmodium vivax               36
Plasmodium falciparum          36
Cyclospora cayetanensis        30
Plasmodium malariae            30
Ascaris lumbricoides           29
Entamoeba histolytica          27
Giardia duodenalis             27
Plasmodium ovale               27
Strongyloides stercoralis      22
Hymenolepis spp                21
Dibothriocephalus latus        21
Trichuris Trichuria            19
Enterobius vermicularis        18
Toxocara canis                 17
Ancyclostoma sp.               17
Sarcocystis sp.                16
Cryptosporidium sp.            16
Plasmodium knowlesi            15
Echinococcus granulosus        12
Fasciola hepatica              11
Dicrocoelium dendriticum        7
Echinococcus multilocularis     4
Strongyloides fuelleborni       3
Name: species, dtype: int64

In [ ]:
13+9+13+8

43

In [4]:
data.species=data.species.replace({"Hymenolepis diminuta":"Hymenolepis spp",
                    "Hymenolepis nana":"Hymenolepis spp",
                    "Taenia solium":"na",
                    "Clonorchis sinensis":"na",
                    "Trichinella sp.":"na",
                    "B. coli":"na",
                    "Toxoplasma gondii":"na",
                    })

In [5]:
data = data[data['species']!='na']

In [6]:
data['phylum'] = data['phylum'].replace({'Roundworms':'Nematoda','Flatworm':'Platyhelminthes'})
data['class'] = data['class'].replace({'\tConoidasida':'Conoidasida','Tapeworms':'Cestoda'})
data['species'] = data['species'].replace({'Entamoeba histolytica]':'Entamoeba histolytica'})
data['form'] = data['form'].replace({'rings':'trophozoites','decorticated egg':'egg'})
data['sample'] = data['sample'].replace({'intestinal tissue':'intestine', 'intestine ':'intestine'})

In [7]:
# data = [x for _,x in data.groupby('form') if len(x)>4]

# data = pd.concat(data,ignore_index=True)
data = [x for _,x in data.groupby('sample') if len(x)>2]

data = pd.concat(data,ignore_index=True)
data = [x for _,x in data.groupby('species') if len(x)>12]

data = pd.concat(data,ignore_index=True)
data['species'] = data['species'].str.replace('.','')


In [11]:
data['genus'].value_counts()

Plasmodium           144
Cyclospora            30
Ascaris               29
Giardia               27
Entamoeba             27
Hymenolepis           21
Dibothriocephalus     21
Strongyloides         20
Trichuris             19
Enterobius            18
Toxocara              17
Ancylostoma           17
Cryptosporidium       16
Sarcocystis           16
Name: genus, dtype: int64

In [9]:
labels =['phylum', 'class', 'genus', 'species', 'form', 'sample']
for col in labels:
  train,test = train_test_split(data.copy(),test_size=0.25,stratify = data[col])
  print(50*'-'+col+50*'-')
  for x in labels:
    print(f"{col} -> {x}")
    print(set(train[x].unique())-set(test[x].unique()))

--------------------------------------------------phylum--------------------------------------------------
phylum -> phylum
set()
phylum -> class
set()
phylum -> genus
set()
phylum -> species
set()
phylum -> form
set()
phylum -> sample
set()
--------------------------------------------------class--------------------------------------------------
class -> phylum
set()
class -> class
set()
class -> genus
set()
class -> species
set()
class -> form
set()
class -> sample
set()
--------------------------------------------------genus--------------------------------------------------
genus -> phylum
set()
genus -> class
set()
genus -> genus
set()
genus -> species
set()
genus -> form
set()
genus -> sample
set()
--------------------------------------------------species--------------------------------------------------
species -> phylum
set()
species -> class
set()
species -> genus
set()
species -> species
set()
species -> form
set()
species -> sample
set()
---------------------------------------

In [10]:
train, val = train_test_split(
    data, test_size=0.25, random_state=2021,stratify = data['genus'])

In [12]:

for path in data['species'].unique():
  os.makedirs(f"/content/train/{path}")


for path in data['species'].unique():
  os.makedirs(f"/content/val/{path}")

In [ ]:
!rm -r train val

In [13]:
t = []
t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomPerspective(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.GaussianBlur(5),
        torchvision.transforms.RandomVerticalFlip(p=1)]))


t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomVerticalFlip(p=1)]))

t.append(torchvision.transforms.Compose([
        torchvision.transforms.Grayscale(3),
        torchvision.transforms.RandomPerspective(p=1)]))

In [ ]:
data.head()

,phylum,class,genus,species,form,sample,image_name,image_url,img_path
0,Nematoda,Chromadorea,Ancylostoma,Ancyclostoma sp,adult,intestine,Aduodenale_head_A.jpg,https://www.cdc.gov//dpdx/hookworm/images/4/Ad...,./Dataset/Ancylostoma/Aduodenale_head_A.jpg
1,Nematoda,Chromadorea,Ancylostoma,Ancyclostoma sp,adult,intestine,Namericanus_head_B.jpg,https://www.cdc.gov//dpdx/hookworm/images/4/Na...,./Dataset/Ancylostoma/Namericanus_head_B.jpg
2,Nematoda,Chromadorea,Ancylostoma,Ancyclostoma sp,adult,intestine,Hookworm_tissue_BAM1.jpg,https://www.cdc.gov//dpdx/hookworm/images/5/Ho...,./Dataset/Ancylostoma/Hookworm_tissue_BAM1.jpg
3,Nematoda,Chromadorea,Ancylostoma,Ancyclostoma sp,adult,intestine,Hookworm_tissue_BAM2.jpg,https://www.cdc.gov//dpdx/hookworm/images/5/Ho...,./Dataset/Ancylostoma/Hookworm_tissue_BAM2.jpg
4,Nematoda,Chromadorea,Ancylostoma,Ancyclostoma sp,adult,intestine,Hookworm_tissue_BAM3.jpg,https://www.cdc.gov//dpdx/hookworm/images/5/Ho...,./Dataset/Ancylostoma/Hookworm_tissue_BAM3.jpg


In [14]:
def transform_save(data,index,folder):
  img = Image.open(data[8])
  images = [img]
  images.extend((T(img) for T in t))
  image_name = data[6].split('.')[0]
  for i, image in enumerate(images):
    image.convert('RGB').save(f"/content/{folder}/{data[3]}/{image_name}_{i}_{index}.jpg")
  values = [[data[0],data[1],data[2],data[3],data[4],data[5],f"./{folder}/{data[3]}/{image_name}_{i}_{index}.jpg"] for i in range(len(images))]
  return values


def get_dataframe(dataframe,folder = 'train'):
  new = []
  for i in dataframe.index:
    new.extend(transform_save(dataframe.loc[i,:].values,index =i,folder=folder))
  return pd.DataFrame(new,columns=['phylum','class','genus','species','form','sample','img_path'])

In [ ]:

def get_val_dataframe(dataframe):
  new = []
  
  for data in dataframe.values:
    new.append([data[0],data[1],data[2],data[3],data[4],data[5],copyfile(data[8],f"./val/{data[3]}/{data[6]}.jpg")])
  return pd.DataFrame(new,columns=['phylum','class','genus','species','form','sample','img_path'])

In [ ]:
data.shape[0]*5/4

555.0

In [15]:
train = get_dataframe(train)

In [16]:
val = get_dataframe(val,folder = 'val')

In [17]:
train.to_csv('/content/train/train.csv',index = False)
val.to_csv('/content/val/val.csv',index = False)

In [18]:
!7z a Train.7z train
!7z a Val.7z val


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive:
  0M Scan           19 folders, 1581 files, 16511906 bytes (16 MiB)

Creating archive: Train.7z

Items to compress: 1600

  0%      5% 146 + train/Ascaris lumbricoides/Ascaris_eggs_tissue_400x_BAM2_1_45.jpg                                                                             13% 247 + train/Cyclospora cayetanensis/Cyclospora_AF_AZ2_2_76.jpg                                                                    21% 378 + train/Dibothriocephalus latus/D . thrium_